<a href="https://colab.research.google.com/github/Christoforos00/NLP_project/blob/main/nlp_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize

corpus = gutenberg.raw('melville-moby_dick.txt')


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokens = word_tokenize(corpus)          #tokenize the whole corpus and find words with under 10 occurancies
counts = nltk.FreqDist(tokens)
under10 = [w for w in counts if counts[w]<=10]
tokens = [token for token in tokens if token not in under10]

In [ ]:
sentences = sent_tokenize(corpus)         #tokenize per sentence
sentences_tokenized = []

for sentance in sentences:
    sent_tok = word_tokenize(sentance)  
    sentences_tokenized.append(sent_tok)


In [ ]:
from random import shuffle                    #shuffle and split the corpus
shuffle(sentences_tokenized)

sentances_num = len(sentences_tokenized)
train_sentances = sentences_tokenized[:int(sentances_num*0.6)]
dev_sentances = sentences_tokenized[int(sentances_num*0.6):int(sentances_num*0.8)]
test_sentances = sentences_tokenized[int(sentances_num*0.8):]


In [ ]:
from collections import Counter         #create the bigram and trigram models
from nltk.util import ngrams

unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sentance in train_sentances:
    
    sentance = [word for word in sentance if word in tokens]

    unigram_counter.update([gram for gram in ngrams(sentance, 1, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ])
    bigram_counter.update([gram for gram in ngrams(sentance, 2, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ])
    trigram_counter.update([gram for gram in ngrams(sentance, 3, pad_left=True, pad_right=True,
                                                   left_pad_symbol='<s>',right_pad_symbol='<e>') ])


In [ ]:
import math          #implement the models (i)


def biGramModel(bigram_counter, unigram_counter, sentences_tokenized, alpha, vocab_size):
    sum_prob = 0
    bigram_cnt = 0
    for sent in sentences_tokenized:
        sent = ['<s>'] + sent + ['<e>']
        for idx in range(1,len(sent)):
            if ( sent[idx-1]=='<s>' ):
                continue
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
            sum_prob += math.log2(bigram_prob)
            bigram_cnt+=1

    CE = -sum_prob / bigram_cnt
    perpl = math.pow(2,CE)
    return CE, perpl


def triGramModel(trigram_counter, bigram_counter, sentences_tokenized, alpha, vocab_size):
    sum_prob = 0
    trigram_cnt = 0

    for sent in sentences_tokenized:
        sent = ['<s>'] + ['<s>'] + sent + ['<e>'] + ['<e>']
        for idx in range(2,len(sent)-1):
            if (sent[idx-2]=='<s>' or sent[idx-1]=='<s>' ):
              continue
            trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] +alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
            sum_prob += math.log2(trigram_prob)
            trigram_cnt+=1

    CE = -sum_prob / trigram_cnt
    perpl = math.pow(2,CE)
    return CE, perpl

def interpModel(trigram_counter, bigram_counter, unigram_counter, sentences_tokenized, alpha, vocab_size, lamda):
    sum_prob = 0
    trigram_cnt = 0

    for sent in sentences_tokenized:
        sent = ['<s>'] + ['<s>'] + sent + ['<e>'] + ['<e>']
        for idx in range(2,len(sent)-1):
            if (sent[idx-2]=='<s>' or sent[idx-1]=='<s>' ):
                continue
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
            trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] +alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
            sum_prob += math.log2(lamda*trigram_prob + (1-lamda) * bigram_prob)
            trigram_cnt+=1

    CE = -sum_prob / trigram_cnt
    perpl = math.pow(2,CE)
    return CE, perpl

In [ ]:
import random

dummy_dev_sentances = []
for sentance in dev_sentances:
  dummy_sentance = [random.sample(tokens,1)[0] for word in sentance]
  dummy_dev_sentances.append(dummy_sentance)

dummy_test_sentances = []
for sentance in test_sentances:
  dummy_sentance = [random.sample(tokens,1)[0] for word in sentance]
  dummy_test_sentances.append(dummy_sentance)


In [ ]:
alpha = 0.01             #compare real vs dummy sentances (ii)

print("Cross Entropy, Perplexity in bigrams")
print ("dev sentances: ", biGramModel(bigram_counter, unigram_counter, dev_sentances, alpha, len(tokens)) )
print ("dummy sentances: ", biGramModel(bigram_counter, unigram_counter, dummy_dev_sentances, alpha, len(tokens)) )

print("\nCross Entropy, Perplexity in trigrams")
print ("dev sentances: ", triGramModel(trigram_counter, bigram_counter, dev_sentances, alpha, len(tokens)) )
print ("dummy sentances: ", triGramModel(trigram_counter, bigram_counter, dummy_dev_sentances, alpha, len(tokens)) )


Cross Entropy, Perplexity in bigrams
dev sentances:  (12.139227780585546, 4510.988149074712)
dummy sentances:  (13.665288119333885, 12991.569569881716)

Cross Entropy, Perplexity in trigrams
dev sentances:  (15.956523655111827, 63590.50101597105)
dummy sentances:  (17.154884977548086, 145926.7635063462)


In [ ]:
        
print("Cross Entropy, Perplexity")       #results in the test sentances (iii)
print ("bigram model: ", biGramModel(bigram_counter, unigram_counter, test_sentances, alpha, len(tokens)) )
print ("trigram model: ", triGramModel(trigram_counter, bigram_counter, test_sentances, alpha, len(tokens)) )


Cross Entropy, Perplexity
bigram model:  (12.126698479234742, 4471.981421325683)
trigram model:  (15.940354884809375, 62881.79955247013)


In [ ]:
for lamda in [0.1,0.3,0.5,0.7,0.9]:            #combinations with interpolation  (iv)
    print ( interpModel(trigram_counter, bigram_counter, unigram_counter, dev_sentances, alpha, len(tokens), lamda))

(12.003485164312425, 4105.906798788918)
(12.129550568523854, 4480.830904793424)
(12.315400941372546, 5096.888018293171)
(12.608172545829257, 6243.643918077654)
(13.228342268113735, 9596.83030741689)


[',']